In [1]:
from facenet_pytorch import InceptionResnetV1, training, fixed_image_standardization
from torch.utils.tensorboard import SummaryWriter
import torch
import torchvision
import pandas as pd
import numpy as np
import os
from PIL import Image

In [2]:
batch_size = 8
epochs = 16
workers = 0 if os.name == 'nt' else 8
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [4]:
data = pd.read_pickle('./race_prediction_image/data.pkl')
data['race'] = np.where(
    data['race'] == 4, 0.0, data['race']
)
data['race'] = np.where(
    data['race'] == 5, np.nan, data['race']
)
data = data.dropna()
print(data['race'].value_counts())
data.head()

0.0    2726
1.0     318
2.0     200
3.0     112
Name: race, dtype: int64


,id,race,img_path,absolute_img_path,cropped_path
0,12488.0,0.0,profile pics/60147.jpeg,./data/profile pics/60147.jpeg,./data/cropped/60147.jpeg
1,719703.0,0.0,profile pics/60148.jpeg,./data/profile pics/60148.jpeg,./data/cropped/60148.jpeg
2,722153.0,3.0,profile pics/60149.jpeg,./data/profile pics/60149.jpeg,./data/cropped/60149.jpeg
5,811618.0,3.0,profile pics/60152.jpeg,./data/profile pics/60152.jpeg,./data/cropped/60152.jpeg
6,822540.0,0.0,profile pics/60153.jpeg,./data/profile pics/60153.jpeg,./data/cropped/60153.jpeg


In [5]:
def load_images(input):
    if os.path.exists(input):
        tmp = Image.open(input)
        test = tmp.getbands()
        keep = tmp.copy()
        return keep
    else: return pd.NA

In [6]:
data['face'] = data['cropped_path'].apply(load_images)
data = data.dropna()
print(data.shape)
data.head()

(2385, 6)


,id,race,img_path,absolute_img_path,cropped_path,face
5,811618.0,3.0,profile pics/60152.jpeg,./data/profile pics/60152.jpeg,./data/cropped/60152.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...
7,865071.0,0.0,profile pics/60154.jpeg,./data/profile pics/60154.jpeg,./data/cropped/60154.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...
8,988211.0,0.0,profile pics/60155.jpeg,./data/profile pics/60155.jpeg,./data/cropped/60155.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...
9,1025311.0,0.0,profile pics/60156.jpeg,./data/profile pics/60156.jpeg,./data/cropped/60156.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...
10,1143891.0,3.0,profile pics/60157.jpeg,./data/profile pics/60157.jpeg,./data/cropped/60157.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...


In [7]:
class Face_Race_Dataset(torch.utils.data.Dataset):
    def __init__(self, x,y):
        self.x = x
        self.y = y
        self.transform = torchvision.transforms.Compose(
                            [
                            np.float32,
                            torchvision.transforms.ToTensor(),
                            fixed_image_standardization])

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return self.transform(self.x[index]),torch.tensor(self.y[index],dtype=torch.long)

In [8]:
totalset = Face_Race_Dataset(data['face'].tolist(),data['race'].tolist())

In [9]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=4
).to(device)

In [11]:
resnet.load_state_dict(torch.load('./race_prediction_image/dataset1_pic_race_model.pt'))
dataloader = torch.utils.data.DataLoader(totalset,batch_size=32,num_workers=workers,shuffle=False)
resnet.eval()
total_pred = []
total_y = []
for i, (x,y) in enumerate(dataloader):
    x = x.to(device)
    preds = resnet(x)
    _, pred_y = torch.max(preds,1)
    pred_y = pred_y.detach().cpu().numpy().tolist()
    y = y.detach().cpu().numpy().tolist()
    total_pred += pred_y
    total_y += y


C:\Users\zhouy\AppData\Local\Temp/ipykernel_13252/2642461197.py:15: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return self.transform(self.x[index]),torch.tensor(self.y[index],dtype=torch.long)


In [12]:
data['pred_race'] = total_pred
data.head()


,id,race,img_path,absolute_img_path,cropped_path,face,pred_race
5,811618.0,3.0,profile pics/60152.jpeg,./data/profile pics/60152.jpeg,./data/cropped/60152.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...,2
7,865071.0,0.0,profile pics/60154.jpeg,./data/profile pics/60154.jpeg,./data/cropped/60154.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...,2
8,988211.0,0.0,profile pics/60155.jpeg,./data/profile pics/60155.jpeg,./data/cropped/60155.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...,0
9,1025311.0,0.0,profile pics/60156.jpeg,./data/profile pics/60156.jpeg,./data/cropped/60156.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...,2
10,1143891.0,3.0,profile pics/60157.jpeg,./data/profile pics/60157.jpeg,./data/cropped/60157.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...,3


In [13]:
from sklearn import metrics
from sklearn.metrics import classification_report
print(classification_report(data['race'], data['pred_race']))

              precision    recall  f1-score   support

         0.0       0.97      0.56      0.71      1951
         1.0       0.40      0.94      0.56       224
         2.0       0.18      0.82      0.29       136
         3.0       0.31      0.42      0.36        74

    accuracy                           0.61      2385
   macro avg       0.46      0.68      0.48      2385
weighted avg       0.85      0.61      0.66      2385



In [14]:
data.head()


,id,race,img_path,absolute_img_path,cropped_path,face,pred_race
5,811618.0,3.0,profile pics/60152.jpeg,./data/profile pics/60152.jpeg,./data/cropped/60152.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...,2
7,865071.0,0.0,profile pics/60154.jpeg,./data/profile pics/60154.jpeg,./data/cropped/60154.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...,2
8,988211.0,0.0,profile pics/60155.jpeg,./data/profile pics/60155.jpeg,./data/cropped/60155.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...,0
9,1025311.0,0.0,profile pics/60156.jpeg,./data/profile pics/60156.jpeg,./data/cropped/60156.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...,2
10,1143891.0,3.0,profile pics/60157.jpeg,./data/profile pics/60157.jpeg,./data/cropped/60157.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...,3


In [15]:
changeName = data.copy()
changeName['race'] = np.where(
    changeName['race'] == 2.0, 5.0, changeName['race']
)
changeName['race'] = np.where(
    changeName['race'] == 3.0, 2.0, changeName['race']
)
changeName['race'] = np.where(
    changeName['race'] == 5.0, 3.0, changeName['race']
)
changeName['pred_race'] = np.where(
    changeName['pred_race'] == 2, 5, changeName['pred_race']
)
changeName['pred_race'] = np.where(
    changeName['pred_race'] == 3, 2, changeName['pred_race']
)
changeName['pred_race'] = np.where(
    changeName['pred_race'] == 5, 3, changeName['pred_race']
)
changeName.head()

,id,race,img_path,absolute_img_path,cropped_path,face,pred_race
5,811618.0,2.0,profile pics/60152.jpeg,./data/profile pics/60152.jpeg,./data/cropped/60152.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...,3
7,865071.0,0.0,profile pics/60154.jpeg,./data/profile pics/60154.jpeg,./data/cropped/60154.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...,3
8,988211.0,0.0,profile pics/60155.jpeg,./data/profile pics/60155.jpeg,./data/cropped/60155.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...,0
9,1025311.0,0.0,profile pics/60156.jpeg,./data/profile pics/60156.jpeg,./data/cropped/60156.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...,3
10,1143891.0,2.0,profile pics/60157.jpeg,./data/profile pics/60157.jpeg,./data/cropped/60157.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...,2


In [16]:
labeled_users = './data/labeled_users.csv'
df_users = pd.read_csv(labeled_users)
df_users.head()

,user_id,is_female,year_born,race
0,12488.0,0.0,1980.0,4.0
1,719703.0,0.0,1985.0,4.0
2,722153.0,1.0,1973.0,3.0
3,749003.0,0.0,1982.0,5.0
4,755531.0,0.0,1982.0,4.0


In [17]:
A = changeName[['id','pred_race']].copy()
B = df_users[['user_id','race']].copy()
B.shape

(4132, 2)

In [18]:
output = B.join(A.set_index('id'), on='user_id')
output.head()

,user_id,race,pred_race
0,12488.0,4.0,NaN
1,719703.0,4.0,NaN
2,722153.0,3.0,NaN
3,749003.0,5.0,NaN
4,755531.0,4.0,NaN


In [20]:
output.to_csv('./race_prediction_image/userid_race_predrace.csv',index=False)